# Uniswap V2
For this strategy, we will provide the liquidity just as in Uniswap v2. We will provide liquidity in the interval [minTick, maxTick].

Import code dependencies

In [1]:
from datetime import date, datetime
import pandas as pd

from demeter import TokenInfo, Actuator, Strategy, RowData, ChainType, MarketInfo, AtTimeTrigger
# perhaphs the one below could be a class?
from demeter.result import performance_metrics, round_results
from demeter.uniswap import UniV3Pool, UniLpMarket

Set pandas output format

In [2]:
pd.options.display.max_columns = None
pd.set_option("display.width", 5000)

Custom Uniswap V2 strategy with add liquidity from tick -887272(minTick) to 887272(maxTick)

In [3]:
class UniswapV2Strategy(Strategy):
    def initialize(self):
        """
        Initialize function will be called right before a backtest start.
        You can do various things here, e.g. register a trigger, or add a simple moving average line.
        """
        new_trigger = AtTimeTrigger(  # define a new trigger
            time=datetime(2023, 8, 15, 12, 0, 0), do=self.work  # we will make the action happen at 12:00 20/8/22
        )  # This is a callback function, defines what to do at this time.
        self.triggers.append(new_trigger)  # Register our trigger

    def work(self, row_data: RowData):
        """
        When time is up, work function will be called.
        """
        lp_market: UniLpMarket = self.markets[market_key]  # pick our market.
        lp_market.add_liquidity_by_tick(-887272, 887272)  # add liquidity
        pass

Main logic to run Actuator, init two token and market with key "market1"

In [4]:
# Declare a token, and it's name will be used as unit of amounts.
usdc = TokenInfo(name="usdc", decimal=6)  # declare token usdc
eth = TokenInfo(name="eth", decimal=18)  # declare token eth
# Declare an Uniswap V3 pool. We will set the parameters according to the real pool on chain.
pool = UniV3Pool(token0=usdc, token1=eth, fee=0.05, quote_token=usdc)

# Declare a market key, which will be used to find the corresponding market in broker
market_key = MarketInfo("market1")
# Declare the market,
market = UniLpMarket(market_key, pool)  # uni_market:UniLpMarket, positions: 0, total liquidity: 0
# load data for market. those data is prepared by download tool
market.data_path = "../data"  # set data path
market.load_data(
    chain=ChainType.polygon.name,  # load data
    contract_addr="0x45dda9cb7c25131df268515131f647d726f50608",
    start_date=date(2023, 8, 15),
    end_date=date(2023, 8, 15),
)

# Declare the Actuator, which controls the whole process
actuator = Actuator()  # declare actuator, Demeter Actuator (broker:assets: ; markets: )
# add market to broker
actuator.broker.add_market(market)
# Initial some fund to broker.
actuator.broker.set_balance(usdc, 10000)
actuator.broker.set_balance(eth, 10)
# Set strategy to actuator
actuator.strategy = UniswapV2Strategy()  # set strategy to actuator
# Set price. Those price will be used in all markets.
# Usually, you will have to find the price list from outer source.
# Luckily, uniswap pool data contains price information. So UniLpMarket provides a function to retrieve price list.
actuator.set_price(market.get_price_from_data())
# run test, If you use default parameter, final fund status will be printed in console.

# actuator.run()

Run actuator with evaluators and save result to files

In [5]:
actuator.account_status_df

""


In [6]:
actuator.run()

2024-10-04 19:43:23,206 - INFO - Qute token is USDC
2024-10-04 19:43:23,207 - INFO - init strategy...
2024-10-04 19:43:23,208 - INFO - start main loop...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1440/1440 [00:00<00:00, 6824.87it/s]
2024-10-04 19:43:23,433 - INFO - main loop finished
2024-10-04 19:43:23,449 - INFO - Print actuator summary
2024-10-04 19:43:23,453 - INFO - Backtesting finished, execute time 0.24803495407104492s


Final account status                              
Token balance in broker       
USDC      :0                        ETH       :4.5592684                
Position value in markets     
market1(UniLpMarket)
token0    :USDC                     token1    :ETH                      fee(%)    :0.0500                   quote token:USDC                     
positions 
   lower_tick  upper_tick     pending0        pending1        liquidity
0     -887270      887270  0.096610981  0.000065859322  233253759509000

Quote by: USDC
Account balance history                           
l1                  net_value tokens             market1                                                                                          price     
l2                              USDC       ETH net_value base_uncollected quote_uncollected base_in_position quote_in_position position_count       ETH USDC
2023-08-15 00:00:00 28436.944  10000        10         0                0                 0                0   

In [7]:
print({k: v for k, v in round_results(performance_metrics(
    actuator.account_status_df["net_value"], 
    benchmark=actuator.account_status_df["price"]["ETH"]
)).items()})


{Start period: Timestamp('2023-08-15 00:00:00'), End period: Timestamp('2023-08-15 23:59:00'), Duration: Timedelta('1 days 00:00:00'), Return: -166.87959, Rate of Return: -0.00587, APR: -0.88332, Max Draw Down: 0.00992, Sharpe Ratio: -8.9171, Volatility: 0.10242, Alpha: -0.25907, Beta: 0.64763, Benchmark return rate: -0.00906, Benchmark APR: -0.9639}


In [8]:
actuator.save_result(
    path="./result",  # save path
    account=True,  # save account status list as a csv file
    actions=True,  # save actions as a json file and a pickle file
)

2024-10-04 19:43:45,755 - INFO - files have saved to ./result/backtest-20241004-194345.account.csv,./result/backtest-20241004-194345.pkl


['./result/backtest-20241004-194345.account.csv',
 './result/backtest-20241004-194345.pkl']

### Let's try it with the newer Ethereum data 

In [9]:
class UniswapV2Strategy(Strategy):
    def initialize(self):
        """
        Initialize function will be called right before a backtest start.
        You can do various things here, e.g. register a trigger, or add a simple moving average line.
        """
        new_trigger = AtTimeTrigger(  # define a new trigger
            time=datetime(2024, 1, 1, 12, 0, 0), do=self.work  # we will make the action happen at 12:00 20/8/22
        )  # This is a callback function, defines what to do at this time.
        self.triggers.append(new_trigger)  # Register our trigger

    def work(self, row_data: RowData):
        """
        When time is up, work function will be called.
        """
        lp_market: UniLpMarket = self.markets[market_key]  # pick our market.
        lp_market.add_liquidity_by_tick(-887272, 887272)  # add liquidity
        pass

Main logic to run Actuator, init two token and market with key "market1"

In [13]:
# Declare a token, and it's name will be used as unit of amounts.
usdc = TokenInfo(name="usdc", decimal=6)  # declare token usdc
eth = TokenInfo(name="eth", decimal=18)  # declare token eth
# Declare an Uniswap V3 pool. We will set the parameters according to the real pool on chain.
pool = UniV3Pool(token0=usdc, token1=eth, fee=0.05, quote_token=usdc)

# Declare a market key, which will be used to find the corresponding market in broker

# are all of the markets just market1, let's make it clear that we can just name it as we want to
market_key = MarketInfo("uniswap")
# Declare the market,
market = UniLpMarket(market_key, pool)  # uni_market:UniLpMarket, positions: 0, total liquidity: 0
# load data for market. those data is prepared by download tool
market.data_path = "/Users/gnapsamuel/Documents/AMM/demeter-fetch/sample-data"  # set data path
market.load_data(
    chain=ChainType.ethereum.name,  # load data
    contract_addr="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",
    start_date=date(2024, 7, 1),
    end_date=date(2024, 9, 30),
)

# Declare the Actuator, which controls the whole process
actuator = Actuator()  # declare actuator, Demeter Actuator (broker:assets: ; markets: )
# add market to broker
actuator.broker.add_market(market)
# Initial some fund to broker.
actuator.broker.set_balance(usdc, 10000)
actuator.broker.set_balance(eth, 10)
# Set strategy to actuator
actuator.strategy = UniswapV2Strategy()  # set strategy to actuator
# Set price. Those price will be used in all markets.
# Usually, you will have to find the price list from outer source.
# Luckily, uniswap pool data contains price information. So UniLpMarket provides a function to retrieve price list.
actuator.set_price(market.get_price_from_data())
# run test, If you use default parameter, final fund status will be printed in console.

# actuator.run()

2024-10-04 19:47:53,143 - INFO - start load files from 2024-07-01 to 2024-09-30...
2024-10-04 19:47:53,686 - INFO - load file complete, preparing...
2024-10-04 19:47:55,016 - INFO - data has been prepared


Run actuator with evaluators and save result to files

In [17]:
actuator.account_status_df

l1                                              net_value tokens       uniswap                                                                                                                      price     
l2                                                          USDC ETH net_value base_uncollected quote_uncollected base_in_position quote_in_position position_count                                   ETH USDC
2024-07-01 00:00:00  44343.933168532335355692139287799831  10000  10     0E-31                0                 0                0                 0              0  3434.3933168532335355692139287799831    1
2024-07-01 00:01:00  44347.367561849188589227708501728644  10000  10     0E-31                0                 0                0                 0              0  3434.7367561849188589227708501728644    1
2024-07-01 00:02:00  44343.933168532335355692139287799831  10000  10     0E-31                0                 0                0                 0              0  3434.3933168532335355692139287799831    1
2024-07-01 00:03:00  44343.933168532335355692139287799831  10000  10     0E-31                0                 0                0                 0              0  3434.3933168532335355692139287799831    1
2024-07-01 00:04:00  44340.499118620473308361303157484093  10000  10     0E-31                0                 0                0                 0              0  3434.0499118620473308361303157484093    1
...                                                   ...    ...  ..       ...              ...               ...              ...               ...            ...                                   ...  ...
2024-09-30 23:55:00  35998.511959309477977927709021285664  10000  10     0E-31                0                 0                0                 0              0  2599.8511959309477977927709021285664    1
2024-09-30 23:56:00  36024.522173719536614560719791130060  10000  10     0E-31                0                 0                0                 0              0  2602.4522173719536614560719791130060    1
2024-09-30 23:57:00  36024.522173719536614560719791130060  10000  10     0E-31                0                 0                0                 0              0  2602.4522173719536614560719791130060    1
2024-09-30 23:58:00  36027.124625936908568222175863109178  10000  10     0E-31                0                 0                0                 0              0  2602.7124625936908568222175863109178    1
2024-09-30 23:59:00  36027.124625936908568222175863109178  10000  10     0E-31                0                 0                0                 0              0  2602.7124625936908568222175863109178    1

[132480 rows x 11 columns]

In [15]:
actuator.run()

2024-10-04 19:47:59,098 - INFO - Qute token is USDC
2024-10-04 19:47:59,099 - INFO - init strategy...
2024-10-04 19:47:59,101 - INFO - start main loop...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 132480/132480 [00:11<00:00, 11650.17it/s]
2024-10-04 19:48:10,474 - INFO - main loop finished
2024-10-04 19:48:11,732 - INFO - Print actuator summary
2024-10-04 19:48:11,885 - INFO - Backtesting finished, execute time 12.789311170578003s


Final account status                              
Token balance in broker       
USDC      :10000                    ETH       :10                       
Position value in markets     
uniswap(UniLpMarket)
token0    :USDC                     token1    :ETH                      fee(%)    :0.0500                   quote token:USDC                     
positions 
Empty DataFrame


Quote by: USDC
Account balance history                           
l1                  net_value tokens       uniswap                                                                                          price     
l2                              USDC ETH net_value base_uncollected quote_uncollected base_in_position quote_in_position position_count       ETH USDC
2024-07-01 00:00:00 44343.933  10000  10         0                0                 0                0                 0              0 3434.3933    1
2024-07-01 00:01:00 44347.368  10000  10         0                0                 0              

In [16]:
print({k: v for k, v in round_results(performance_metrics(
    actuator.account_status_df["net_value"], 
    benchmark=actuator.account_status_df["price"]["ETH"]
)).items()})


{Start period: Timestamp('2024-07-01 00:00:00'), End period: Timestamp('2024-09-30 23:59:00'), Duration: Timedelta('92 days 00:00:00'), Return: -8316.80854, Rate of Return: -0.18755, APR: -0.56135, Max Draw Down: 0.3209, Sharpe Ratio: -1.21239, Volatility: 0.48775, Alpha: -0.08159, Beta: 0.7191, Benchmark return rate: -0.24216, Benchmark APR: -0.66716}


In [18]:
actuator.save_result(
    path="./result",  # save path
    account=True,  # save account status list as a csv file
    actions=True,  # save actions as a json file and a pickle file
)

2024-10-04 19:49:45,040 - INFO - files have saved to ./result/backtest-20241004-194944.account.csv,./result/backtest-20241004-194944.pkl


['./result/backtest-20241004-194944.account.csv',
 './result/backtest-20241004-194944.pkl']